In [1]:
import pandas as pd 
import numpy as np 
import rasterio 
from tqdm import tqdm
import os
from preprocessing import NDVIPreprocessing

In [2]:
LIST_FOLDER =  ['dakya_gialai','haiduong','hue','hungson_processing','huy_longan',
                'kienxuong_thaibinh','kimson_ninhbinh','longmy_haugiang','namdinh']

ROOT_PATH = 'D:/Streamlit/api/assets/img'

# Sentinel 2 NDVI 

In [ ]:
# Kiểm tra dữ liệu ngoại lai 
def check_outliers(list_ndvi):
    alpha = beta = 0.15
    gamma = -0.05
    for t in range(1, len(list_ndvi)-1):
        if list_ndvi[t-1] is None:
            continue
        if list_ndvi[t-1] is not None and list_ndvi[t] is not None and list_ndvi[t+1] is not None:
            c_alpha = np.abs(list_ndvi[t] - list_ndvi[t-1])
            c_beta = np.abs(list_ndvi[t+1] - list_ndvi[t])
            c_gamma = np.abs(list_ndvi[t+1] - list_ndvi[t-1])
            print(f"a = {c_alpha},  b = {c_beta}, g = {c_gamma}")
            if c_alpha >= alpha and c_beta >= beta and c_gamma >= gamma:
                list_ndvi[t] = None
    return list_ndvi

test_list = [0.6, 0.55, 0.1, 0.48, 0.4, 0]
test_list = check_outliers(test_list)
print(test_list)


In [56]:
def count_pixel_cloud(data):
    min = 100
    index = None
    for i in range(0, len(data)):
        num_nan = np.isnan(data[i]).sum()
        total_pixel = data[i].shape[0] * data[i].shape[1]
        ratio_cloud = float(num_nan/total_pixel)
        if ratio_cloud < min:
            min = ratio_cloud
            index = i
    return min, index

In [60]:
arr_full = []
range_time = 4
print('Start process')

for name in LIST_FOLDER:
    arr = np.load(f'D:/Streamlit/api/assets/np/{name}.npy')
    # Truy cập vào từng ảnh trong 1 vùng
    print(f'Khu vực {name}')
    area_ndvi_album = []
    for img_index in tqdm(range(0, arr.shape[0], range_time), desc='Processing: '):        
        # Lấy 4 ảnh 1 lần (theo tháng)
        series_img = arr[img_index:img_index + range_time, :, :]
        
        # Tìm ảnh có ít pixel mây nhất và lấy làm ảnh gốc
        min_cloud_pixels, index = count_pixel_cloud(series_img)
        image_fill = series_img[index, :, :]
        # Bù các pixel thiếu bằng cách lấy giá trị từ các ảnh khác trong chuỗi
        for x in range(arr.shape[1]):
            for y in range(arr.shape[2]):
                if np.isnan(image_fill[x, y]):  
                    for i in range(series_img.shape[0]):  
                        if i != index and not np.isnan(series_img[i, x, y]):
                            image_fill[x, y] = series_img[i, x, y]
                            break  
    
        area_ndvi_album.append(image_fill)

    arr_full.append(np.array(area_ndvi_album))
    
print('End process!!!')


Start process
Khu vực dakya_gialai


Processing:   0%|          | 0/53 [00:00<?, ?it/s]

Processing: 100%|██████████| 53/53 [00:00<00:00, 54.04it/s]


Khu vực haiduong


Processing: 100%|██████████| 53/53 [00:01<00:00, 28.52it/s]


...

Khu vực longmy_haugiang


Processing: 100%|██████████| 53/53 [00:01<00:00, 44.32it/s]


Khu vực namdinh


Processing: 100%|██████████| 53/53 [00:04<00:00, 11.67it/s]

End process!!!


In [83]:
process = NDVIPreprocessing()

In [84]:
def load_sentinel1():
    res = []
    for name in LIST_FOLDER: 
        path = f'{ROOT_PATH}/{name}/S1GRD'
        print(path)
        folder = []
        for image_path in tqdm(os.listdir(path), desc='Processing: '):
            img = process.make_S1_raster(f'{path}/{image_path}')
            folder.append(img)
        res.append(np.array(folder))
    return res

res = load_sentinel1()

D:/Streamlit/api/assets/img/dakya_gialai/S1GRD


Processing: 100%|██████████| 209/209 [00:02<00:00, 73.98it/s] 


D:/Streamlit/api/assets/img/haiduong/S1GRD


Processing: 100%|██████████| 209/209 [00:02<00:00, 88.48it/s] 


D:/Streamlit/api/assets/img/hue/S1GRD


...

Processing: 100%|██████████| 209/209 [00:02<00:00, 75.42it/s]


D:/Streamlit/api/assets/img/longmy_haugiang/S1GRD


Processing: 100%|██████████| 209/209 [00:02<00:00, 80.72it/s]


D:/Streamlit/api/assets/img/namdinh/S1GRD


Processing: 100%|██████████| 209/209 [00:02<00:00, 75.86it/s]


In [11]:
arr_full[0][0].shape

(163, 108)

In [12]:
arr_full[0][0:4].shape

(4, 163, 108)

In [85]:
VH = []
VV = []
print('---------- Start Processing ----------')
for area_index in range(len(LIST_FOLDER)):
    list_VH_area_image = []
    list_VV_area_image = []
    print(f'Starting Loading Sentinel 1 GRD at {LIST_FOLDER[area_index]}')
    for image in tqdm(os.listdir(f'{ROOT_PATH}/{LIST_FOLDER[area_index]}/S1GRD'),desc='Processing: '):
        path_to_image=f'{ROOT_PATH}/{LIST_FOLDER[area_index]}/S1GRD/{image}'
        VV_band, VH_band = process.make_S1_raster(path_to_image)
        list_VH_area_image.append(VH_band)
        list_VV_area_image.append(VV_band)
    VH.append(np.array(list_VH_area_image))
    VV.append(np.array(list_VV_area_image))    

print('Done all')

---------- Start Processing ----------
Starting Loading Sentinel 1 GRD at dakya_gialai


Processing: 100%|██████████| 209/209 [00:00<00:00, 327.86it/s]


Starting Loading Sentinel 1 GRD at haiduong


Processing: 100%|██████████| 209/209 [00:00<00:00, 330.60it/s]


Starting Loading Sentinel 1 GRD at hue


...

Starting Loading Sentinel 1 GRD at longmy_haugiang


Processing: 100%|██████████| 209/209 [00:00<00:00, 339.37it/s]


Starting Loading Sentinel 1 GRD at namdinh


Processing: 100%|██████████| 209/209 [00:00<00:00, 349.42it/s]

Done all


In [86]:
VH[0][0:4,:,:].shape[2]

108

In [87]:
VH_full = []
VV_full = []
range_time = 4
print('Start process Sentinel 1 GRD:')

for area_index in range(len(LIST_FOLDER)):
    # Truy cập vào từng ảnh trong 1 vùng
    print(f'Khu vực {LIST_FOLDER[area_index]}')
    area_vh_album = []
    area_vv_album = []
    print(VH[area_index].shape)
    for img_index in tqdm(range(0,VH[area_index].shape[0],range_time),desc='Processing: '):        
        # Lấy 4 ảnh 1 lần (theo tháng)
        series_VH_img = VH[area_index][img_index:img_index + range_time,:,:]
        series_VV_img = VV[area_index][img_index:img_index + range_time,:,:]
        
        # Truy cập vào từng pixel
        image_VH_fill = np.full((series_VH_img.shape[1], series_VH_img.shape[2]), np.nan)
        image_VV_fill = np.full((series_VH_img.shape[1], series_VH_img.shape[2]), np.nan)
        
        for x in range(series_VH_img.shape[1]):
            for y in range(series_VH_img.shape[2]):
                vh_last_not_null = None
                vv_last_not_null = None
                for i in range(series_VH_img.shape[0]):
                    if np.isnan(series_VH_img[i,x,y]):
                        continue
                    elif np.isnan(series_VV_img[i,x,y]):
                        continue
                    else:
                        vh_last_not_null = series_VH_img[i,x,y]
                        vv_last_not_null = series_VV_img[i,x,y]
                image_VH_fill[x,y] = vh_last_not_null
                image_VV_fill[x,y] = vv_last_not_null
       
        area_vh_album.append(image_VH_fill)
        area_vv_album.append(image_VV_fill)
        
    VH_full.append(np.array(area_vh_album))
    VV_full.append(np.array(area_vv_album))

print('Done process!!!')

Start process Sentinel 1 GRD:
Khu vực dakya_gialai
(209, 163, 108)


Processing: 100%|██████████| 53/53 [00:08<00:00,  6.59it/s]


Khu vực haiduong
(209, 129, 191)


Processing: 100%|██████████| 53/53 [00:11<00:00,  4.62it/s]


Khu vực hue
(209, 215, 219)


...

Khu vực longmy_haugiang
(209, 161, 135)


Processing: 100%|██████████| 53/53 [00:09<00:00,  5.39it/s]


Khu vực namdinh
(209, 238, 249)


Processing: 100%|██████████| 53/53 [00:26<00:00,  1.98it/s]

Done process!!!


In [107]:
IW_ratio = []  # VV/VH
IW_cross = []  # VH - VV

for area_index in range(len(LIST_FOLDER)):
    VH_area_album = VH_full[area_index]
    VV_area_album = VV_full[area_index]
    IW_ratio_area = []
    IW_cross_area = []
    for i in tqdm(range(len(VH_area_album)), desc='Calculating: '):
        imgVH = VH_area_album[i].astype(np.float64)  # Chuyển đổi sang float64
        imgVV = VV_area_album[i].astype(np.float64)  # Chuyển đổi sang float64
        # Sử dụng np.divide với điều kiện VH != 0
        img_IW_ratio = np.divide(imgVV, imgVH, out=np.zeros_like(imgVV, dtype=np.float64), where=imgVH != 0)
        img_IW_cross = imgVH - imgVV
        
        IW_ratio_area.append(img_IW_ratio)
        IW_cross_area.append(img_IW_cross)
        
    IW_ratio.append(np.array(IW_ratio_area))
    IW_cross.append(np.array(IW_cross_area))


Calculating: 100%|██████████| 53/53 [00:00<00:00, 2216.71it/s]


In [61]:
arr_ =[]
for index_ in range(len(LIST_FOLDER)):
    arr = arr_full[index_]
    arr_ratio = []
    for i in tqdm(range(arr.shape[0])):
        num_nan = np.isnan(arr[i]).sum()

        # Tính tổng số lượng phần tử trong mảng
        total_elements = arr.shape[1] * arr.shape[2]
        arr_ratio.append(float(num_nan/total_elements) * 100)
    arr_.append(arr_ratio)

100%|██████████| 53/53 [00:00<00:00, 13285.81it/s]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dữ liệu ví dụ cho 9 biểu đồ cột
data =  arr_ # 9 bộ dữ liệu, mỗi bộ có 5 giá trị

# Tạo figure và lưới các subplots 3x3
fig, axes = plt.subplots(9, 1, figsize=(30, 40))

# Duyệt qua các axes và vẽ biểu đồ cột
for i, ax in enumerate(axes.flat):
    ax.bar(range(len(data[i])), data[i])
    ax.set_title(f'Vùng {LIST_FOLDER[i]}')
    ax.set_xlabel('Tuần')
    ax.set_ylabel('Phần trăm mây')

# Tự động căn chỉnh layout để tránh bị trùng lặp nội dung
plt.tight_layout()
plt.show()

In [ ]:
# Vẽ mảng 2D dưới dạng hình ảnh
import matplotlib.pyplot as plt 

# Tạo bảng 2x5 để hiển thị 10 ảnh
fig, axs = plt.subplots(6, 10, figsize=(30, 30))

# Vẽ từng ảnh trong bảng
for i, ax in enumerate(axs.flat):
    ax.imshow(arr_full[8][i,:,:], cmap='viridis', vmax=1, vmin=-1)  
    ax.set_title(f'Month {i+1}')
    ax.axis('off')  # Ẩn các trục

# Thêm khoảng cách giữa các ô
plt.tight_layout()
plt.show()

In [81]:
dakya = np.load('D:/Streamlit/api/assets/np/namdinh.npy')

In [ ]:
# Vẽ mảng 2D dưới dạng hình ảnh
import matplotlib.pyplot as plt 

# Tạo bảng 2x5 để hiển thị 10 ảnh
fig, axs = plt.subplots(21, 10, figsize=(30, 30))

# Vẽ từng ảnh trong bảng
for i, ax in enumerate(axs.flat):
    ax.imshow(dakya[i,:,:], cmap='viridis', vmax=1, vmin=-1)  
    ax.set_title(f'Week {i+1}')
    ax.axis('off')  # Ẩn các trục

# Thêm khoảng cách giữa các ô
plt.tight_layout()
plt.show()

In [142]:
def normalize_array(array):
    """
    Chuẩn hóa một mảng về khoảng giá trị [0, 1]
    """
    min_val = np.nanmin(array)
    max_val = np.nanmax(array)
    return (array - min_val) / (max_val - min_val)

X_time_series = []
y_time_series = []
for area_index in range(len(LIST_FOLDER)):
    VV_image = VV_full[area_index]
    VH_image = VH_full[area_index]
    IW_cross_image = IW_cross[area_index]
    IW_ratio_image = IW_ratio[area_index]
    NDVI_image = arr_full[area_index]
    
    # Chuẩn hóa toàn bộ hình ảnh
    VV_image = normalize_array(VV_image)
    VH_image = normalize_array(VH_image)
    IW_cross_image = normalize_array(IW_cross_image)
    IW_ratio_image = normalize_array(IW_ratio_image)
    NDVI_image = normalize_array(NDVI_image)
    
    area_series = []
    y_ = []
    for i in tqdm(range(VH_image.shape[0])):
        image_series = []
        y__ = []
        for x in range(VH_image.shape[1]):
            for y in range(VH_image.shape[2]):
                vv = VV_image[i, x, y]
                vh = VH_image[i, x, y]
                iw_cross = IW_cross_image[i, x, y]
                iw_ratio = IW_ratio_image[i, x, y]
                ndvi = NDVI_image[i, x, y]
                image_series.append(np.array([vv, vh, iw_cross, iw_ratio, ndvi])) 
                y__.append(np.array(ndvi))
        area_series.append(np.array(image_series))
        y_.append(y__)
        
    X_time_series.append(np.array(area_series))
    y_time_series.append(np.array(y_))

100%|██████████| 53/53 [00:07<00:00,  6.94it/s]


In [146]:
area_1 = X_time_series[0].transpose(1, 0, 2)
area_1_y = y_time_series[0].transpose(1,0)

In [147]:
area_1.shape

(17604, 53, 5)

In [148]:
area_1_y.shape

(17604, 53)

In [131]:
np.isnan(area_1[8]).sum()

5

In [173]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import tensorflow as tf
from tensorflow.keras import (
    Model,
    Sequential,
    losses,
    optimizers,
    metrics,
    layers,
    initializers,
)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard,
    LearningRateScheduler,
)
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import itertools
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler, ModelCheckpoint

class NDVI_Reconstruction:
    def __init__(self, n_timesteps, n_outputs, n_features):
        self.n_timesteps = n_timesteps
        self.n_outputs = n_outputs
        self.n_features = n_features

    def mse_custom(self, y_true, y_pred):
        err = tf.where(y_true > 0.0, y_true - y_pred, 0)
        return K.mean(K.square(err), axis=-1)

    def mae_custom(self, y_true, y_pred):
        err = tf.where(y_true > 0.0, y_true - y_pred, 0)
        return K.mean(K.abs(err), axis=-1)
    
    def add_CNN_block_1D(self, x_inp, filters, kernel_size=3, padding="same", strides=1):
        x = layers.Conv1D(filters,kernel_size,padding=padding, strides=strides,
                      kernel_initializer=initializers.glorot_normal())(x_inp)
        x = layers.Activation('relu')(x)
        return x
    
    def attention_seq(query_value, scale):
        query,value = query_value
        score = tf.matmul(query, value, transpose_b=True) #(batch, timestamp, 1)
        score = scale * score
        score = tf.nn.softmax(score, axis=1)
        score = score * query 
        
        return score
    
    def fusion_model(self, attention=False, cnn_layers=[8,16], pool_size=2, fcl_size=[16,16], lstm_units=32):
        inputs = list([])
        k = list([])
        var = np.array(['vv', 'vh', 'vv_div_vh', 'vh_minus_vv', 'ndvi'])

        for v in var:
            x_inp = layers.Input(shape=(self.n_timesteps, 1), name='{}_input'.format(v))
            inputs.append(x_inp)
            if v == 'ndvi':
                x_inp = layers.Masking(mask_value=-100)(x_inp)
            x = self.add_CNN_block_1D(x_inp, filters=cnn_layers[0])
            x = layers.Dropout(0.2)(x)
            for f in cnn_layers[1:]:
                x = self.add_CNN_block_1D(x, f)
                x = layers.Dropout(0.2)(x)

            x = layers.MaxPooling1D(pool_size=pool_size, strides=None)(x)
            x = layers.Dropout(0.5)(x)
            x = layers.Flatten()(x)
            for f in fcl_size[:-1]:
                x = layers.Dense(f, activation='relu', kernel_initializer=initializers.glorot_normal())(x)
            k.append(x)

        m = layers.Concatenate()(k)
        m = layers.RepeatVector(self.n_outputs)(m)  # Ensure this is applied correctly

        if attention:
            seq, state, _ = layers.LSTM(lstm_units, activation='relu', return_sequences=True, return_state=True)(m)
            att = tf.keras.layers.Lambda(self.attention_seq, arguments={'scale': 0.01})([seq, tf.expand_dims(state, 1)])
            m = layers.LSTM(lstm_units, activation='relu', return_sequences=True)(att)
        else:
            m = layers.Bidirectional(layers.LSTM(lstm_units, activation='relu', return_sequences=False))(m)
            m = layers.RepeatVector(self.n_outputs)(m)
            m = layers.Bidirectional(layers.LSTM(lstm_units, activation='relu', return_sequences=True))(m)

        m = layers.TimeDistributed(layers.Dense(fcl_size[-1], activation='relu', kernel_initializer=initializers.glorot_normal()))(m)
        out = layers.TimeDistributed(layers.Dense(1, activation='sigmoid', kernel_initializer=initializers.glorot_normal()))(m)

        model = Model(inputs=inputs, outputs=out)
        return model

        
    def config_model(self):
        param_grid = {'cnn_layers': [[8,16]],
            'attention': [False],
            'pool_size':[3],
            'fcl_size':[[32,32]],
            'lstm_units':[16]}

        keys, values = zip(*param_grid.items())
        permutations_params = [dict(zip(keys, v)) for v in itertools.product(*values)]


        params = permutations_params[0]
        attention = params['attention']
        cnn_layers = params['cnn_layers']
        pool_size = params['pool_size']
        fcl_size = params['fcl_size']
        lstm_units = params['lstm_units']
    
    
        fusion_model = self.fusion_model(attention=attention, cnn_layers=cnn_layers, pool_size=pool_size, fcl_size=fcl_size, lstm_units=lstm_units)
        
        return fusion_model
    
    # Hàm scheduler để điều chỉnh learning rate
    def scheduler(self, epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * 0.1

    # Sử dụng hàm scheduler với LearningRateScheduler
    change_lr = LearningRateScheduler(scheduler)    
    def training_model(self, X_new_train, y_train, X_new_val, y_val, masks_fused_train, masks_fused_val):
        fusion_model = self.config_model()
        fusion_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
        
        batch_size = 1024
        epochs = 100
        ver = 0
        initial_lr = 0.005
        
        model_path = 'models/fusion_model.weights.h5'
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=ver)

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=ver, patience=7)
        change_lr = LearningRateScheduler(self.scheduler)
        mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', 
                            verbose=ver, save_best_only=True, save_weights_only=True)

        fusion_model.fit(X_new_train, y_train.reshape(-1, self.n_timesteps, 1),
                        validation_data=(X_new_val, y_val.reshape(-1, self.n_timesteps, 1), masks_fused_val),
                        batch_size=batch_size,
                        sample_weight=masks_fused_train,
                        epochs=epochs,
                        verbose=True,
                        callbacks=[mc, reduce_lr, es, change_lr]
                        )

In [174]:
n_timesteps, n_features, n_outputs  = area_1.shape[1],area_1.shape[2],area_1.shape[1]
object_rec = NDVI_Reconstruction(n_timesteps=n_timesteps, n_features=n_features, n_outputs=n_outputs)
model = object_rec.config_model()
model.summary()

d:\Streamlit\streamlit-env\Lib\site-packages\keras\src\layers\layer.py:915: UserWarning: Layer 'conv1d_18' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ndvi_input          │ (None, 53, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vv_input            │ (None, 53, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vh_input            │ (None, 53, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vv_div_vh_input     │ (None, 53, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vh_minus_vv_input   │ (None, 53, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_1 (Masking) │ (None, 53, 1)     │          0 │ ndvi_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 53, 8)     │         32 │ vv_input[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 53, 8)     │         32 │ vh_input[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 53, 8)     │         32 │ vv_div_vh_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_16 (Conv1D)  │ (None, 53, 8)     │         32 │ vh_minus_vv_inpu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 53, 8)     │         32 │ masking_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 53, 8)     │          0 │ conv1d_10[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 53, 8)     │          0 │ conv1d_12[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_14       │ (None, 53, 8)     │          0 │ conv1d_14[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_16       │ (None, 53, 8)     │          0 │ conv1d_16[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 53, 8)     │          0 │ conv1d_18[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 53, 8)     │          0 │ activation_10[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_18          │ (None, 53, 8)     │          0 │ activation_12[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_21          │ (None, 53, 8)     │          0 │ activation_14[0]

 Total params: 75,857 (296.32 KB)

 Trainable params: 75,857 (296.32 KB)

 Non-trainable params: 0 (0.00 B)